## new.py, named entity detection
To detect name entities we check if they are uppper case also use word tagas to see whether they are proper nouns or not.

In [ ]:
# nltk library is used for text processing
from nltk import corpus
import nltk
from nltk.corpus import wordnet
from nltk.corpus import words as wn
from nltk.corpus import stopwords
# Tokenizers divide strings into lists of substrings.  For example,
# tokenizers can be used to find the words and punctuation in a string.
from nltk import word_tokenize

In [ ]:
# lematizer:
# car, cars, car's, cars' --> car
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
# read the file and tokenize it
file=open("caeser.txt",'r')
text=file.read()
sentences=nltk.sent_tokenize(text)

In [ ]:
# 
named=set()
for sent in sentences:
    
    if sent.isupper():
        named.add(sent)
    # pos-tag is process of classifying words into their parts of speech (Noun, adjective etc)
    pos=nltk.pos_tag(word_tokenize(sent))
    
    for i in range(0,len(pos)):
#      case that named-entity is two words, check if they have upper case letters
        if pos[i][0].isupper() and pos[i+1][0].isupper() and len(pos[i][0])>1 and len(pos[i+1][0])>1 and pos[i+1][1]=="NNP":
            named.add(pos[i][0]+" "+pos[i+1][0])
            i=i+1
#       if it is one word, check upper case, also check if it is proper noun
        elif pos[i][0].isupper() and pos[i][1]=="NNP" and len(pos[i][0])>1:
            named.add(pos[i][0])
        elif pos[i][0][0].isupper() and pos[i][1] == "NNP" and len(pos[i][0])>1:
            named.add(pos[i][0])

In [ ]:
invalidChars = set(string.punctuation.replace("_", ""))
for word in named:
    #print(type(word))
    if not any(char in invalidChars for char in word):
        if word.isupper():
            words.add(word.lower())
        elif word[0].isupper():
            if not (wordnet_lemmatizer.lemmatize(word.lower()) in wn.words()):
                if len(wordnet.synsets(wordnet_lemmatizer.lemmatize(word.lower())))==0:
                    words.add(word.lower())

In [ ]:
for item in list(words):
  thefile.write("%s\n" % item)

## slicing.py 
To discover relations between named-entities, we decided to use graph algorithms and visulisations. Vertices are named-entities and edges represent whether two named-entities named-entities appear together for a given distance to each other. And edge weight is number of occurances.

In [ ]:
#-*-coding:utf-8-*-
import string
import json
from nltk import stem,word_tokenize
from itertools import product
from string import punctuation

In [ ]:
# create a dictionary as a word histogram, keys are name entities
def words2dict(filename,a_dict):
    thefile = open(filename, 'r')
    lines=thefile.readlines()
    for word in lines:
        a_dict[word[:-1]]=a_dict.get(word,0)+1

    thefile.close()
    return a_dict

In [ ]:
# from given text, create a graph,
# edges represents two vertices (words)appearing together within range of slice_len
# graph dictionary:{("daniel,mark"):1,}
# Mike is connected to mark and daniel, they appear n times together

def text2graph(filename,slice_len,graph_dict,words_hash):

    thefile=open(filename,'r')
#   list of punctuations to remove them later
    punctuation=list(string.punctuation)

    line=thefile.readline()
    word_slice=[]
    word_count=0
    total_count=0
    while (line!=""):
        total_count+=1
#    print progress of loop
        if (total_count%1000)==0:
            print total_count
#       tokenize words
        words=word_tokenize(line)
        for word in words:
            if word in punctuation:
                continue
            word_count +=1
            # lowercase the word
            word=word.lower()
            # remove affixes
            word=stem.WordNetLemmatizer().lemmatize(word)
#           if word is one of the name entities
            if (words_hash.get(word,None)!=None):
                word_slice.append(word)
#           if we have enough number of words
            if word_count==slice_len:
                if len(word_slice)>1:
                    
#                   create groups of 2 from all name entities in the slice
#                   and place them on the graph
                    for r in product(word_slice, word_slice):
                        r=tuple(sorted(list(r)))
                        graph_dict[r]=graph_dict.setdefault(r,0)+1
                word_count=0
                word_slice=[]
        line = thefile.readline().decode('utf-8')

    thefile.close()

    return graph_dict


In [ ]:
# file with name entities
words_filename="macbeth-named.txt"
# entire text
text_filename="macbeth.txt"
# slice length parameter
slice_len=10
# graph and name_entity histogram dictionaries
graph_dict={}
words_hash={}
words_hash=words2dict(words_filename,words_hash)
graph_dict=text2graph(text_filename,slice_len,graph_dict,words_hash)

In [ ]:
#  Changing data structure of graph for data processing and visulisation

# list of name entities 
names=[]
names_dict={}
for item in graph_dict.keys():
    names.append(item[0])
    names.append(item[1])

# generate keys for each name entity
names=list(set(names))
idx=[]
for i,k in enumerate(names):
    idx.append((i,k))
    names_dict[k]=i

# save them as csv files
ofile  = open('id_name.csv', "wb")
for (i,item) in idx:
    ofile.write("%d,%s"%(i,item))
    ofile.write("\n")
ofile.close()

ofile  = open('edges.csv', "wb")

for item in graph_dict.items():
    ofile.write("%d,%d,%d"%(names_dict[item[0][0]],names_dict[item[0][1]],item[1]))
    ofile.write("\n")

ofile.close()

### Slicing2.py 
When we pick our text source as Shakespeare plays, we decided to use a better heuristic than slice lenght. Because they have dialogs, we decided to use length of a dialog as our slice length. So this new slicing script gets text, and generates graphs from this information.

In [ ]:
import csv
import itertools

file=open("caeser.txt")
text=file.read()

with open("caeser-named.txt") as f:
    text1=f.readlines()
    
names = [x.strip() for x in text1]
names_dict=[]
g={}
count=0
acts=text.split('\n\n')

for name in names:
    g={}
    g["id"]=count
    g["label"]=name
    names_dict.append(g)
    count=count+1
edges_dict=[]
ed={}

l=list(itertools.combinations(range(len(names_dict)), 2))
for pair in l:
    ed={}
    ed["source"]=pair[0]
    ed["target"]=pair[1]
    ed["weight"]=0
    edges_dict.append(ed)

for pair in edges_dict:
    for act in acts:
        s1=act.lower().count(names_dict[pair["source"]]["label"])
        s2=act.lower().count(names_dict[pair["target"]]["label"])
        pair["weight"]=pair["weight"]+min(s1,s2)
        
# save edges that are 
final_edge=[]
for edge in edges_dict:
    #print(edge)
    if edge['weight']!=0:
        final_edge.append(edge)
# print(final_edge)



In [ ]:
# save results
keys = names_dict[0].keys()
with open('id_name1.csv', 'w',newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(names_dict)

keys = final_edge[0].keys()
with open('edges1.csv', 'w',newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(final_edge)